# Построение рекомендательного сервиса

### Из чего состоит:

Типичный пайплайн рекомендательной системы:

- у сервиса есть база данных(фильмы, музыка, статьи, итд)
есть пользователь
- вся база данных ранжируется по пользователю
- в виде рекомендации пользователю предлагается контент с наивысшим рейтингом


In [23]:
!pip install wget

import os # work with filesystem
import re 
import math
import numpy as np
import wget        # library for URL-downloads
import nltk
import csv

import numpy as np
import pandas as pd

from scipy.spatial import distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Create movie recomendation system

In [24]:
data_status = os.path.exists('./movie_metadata.csv')

if data_status == False:
    url = 'https://drive.google.com/uc?export=view&id=18HGe2_fWzH8sG-BpHmz76IGEp6VsqCq1'
    wget.download(url, 'movie_metadata.csv')

In [25]:
movie_dataset = pd.read_csv('movie_metadata.csv', delimiter=',')

In [26]:
movie_dataset

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


In [27]:
our_movie_dataset = movie_dataset[['movie_title', 'genres', 'content_rating', 'color', 'duration', 'language', 'imdb_score', 'plot_keywords']]
our_movie_dataset = our_movie_dataset.dropna()
data_len = len(our_movie_dataset)

our_movie_dataset['color'] = our_movie_dataset['color'].str.lower()
our_movie_dataset['color'] = our_movie_dataset['color'].str.replace('black and white', 'baw')
our_movie_dataset['language'] = our_movie_dataset['language'].str.lower()
our_movie_dataset['genres'] = our_movie_dataset['genres'].str.lower()
our_movie_dataset['genres'] = our_movie_dataset['genres'].str.replace('|', ' ')
our_movie_dataset['content_rating'] = our_movie_dataset['content_rating'].str.lower()
our_movie_dataset['plot_keywords'] = our_movie_dataset['plot_keywords'].str.replace('|', ' ')
our_movie_dataset = our_movie_dataset.reindex(range(data_len),method='bfill')
our_movie_dataset

<ipython-input-27-247894c69d7f>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  our_movie_dataset['genres'] = our_movie_dataset['genres'].str.replace('|', ' ')
<ipython-input-27-247894c69d7f>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  our_movie_dataset['plot_keywords'] = our_movie_dataset['plot_keywords'].str.replace('|', ' ')


,movie_title,genres,content_rating,color,duration,language,imdb_score,plot_keywords
0,Avatar,action adventure fantasy sci-fi,pg-13,color,178.0,english,7.9,avatar future marine native paraplegic
1,Pirates of the Caribbean: At World's End,action adventure fantasy,pg-13,color,169.0,english,7.1,goddess marriage ceremony marriage proposal pi...
2,Spectre,action adventure thriller,pg-13,color,148.0,english,6.8,bomb espionage sequel spy terrorist
3,The Dark Knight Rises,action thriller,pg-13,color,164.0,english,8.5,deception imprisonment lawlessness police offi...
4,John Carter,action adventure sci-fi,pg-13,color,132.0,english,6.6,alien american civil war male nipple mars prin...
...,...,...,...,...,...,...,...,...
4643,The Quiet,drama thriller,r,color,91.0,english,6.4,deaf death dysfunctional family godparent secret
4644,Circumstance,drama,r,color,107.0,persian,5.9,closeted lesbian iranian islamophobia propagan...
4645,Fruitvale Station,biography drama romance,r,color,85.0,english,7.5,22 year old b.a.r.t. bay area birthday police
4646,The Brass Teapot,comedy fantasy thriller,r,color,101.0,english,6.4,mini dress mini skirt money voyeur voyeurism


In [28]:
preferences = {'movie_title' : '', 'genres' : 'thriller', 'content_rating' : 'pg-13', 'color' : 'color', 'duration' : 100, 'language' : 'english', 'imdb_score' : 10, 'plot_keywords' : 'future game'}

need_to_fill = True

if need_to_fill:

  answer = input('What is your prefered film genre? : ' )
  while not answer.replace(' ','').isalpha():
    answer = input('Please try again : ' )
  preferences['genres'] = answer.lower()

  answer = input('What is your prefered content rating? : ' )
  preferences['content_rating'] = answer.lower()

  answer = input('Do you prefer to watch Black and White films more than Color films? : ' )
  while answer.lower() != 'yes' and answer.lower() != 'no':
    answer = input('Please try again : ' )
  if answer.lower() == 'yes':
    preferences['color'] = 'baw'
  else:
    preferences['color'] = 'color'

  answer = input('What is your oprimal film duration in minutes? : ' )
  while not answer.isnumeric():
    answer = input('Please try again : ' )
  preferences['duration'] = int(answer)

  answer = input('What is your prefered film language? : ' )
  while not answer.replace(' ','').isalpha():
    answer = input('Please try again : ' )
  preferences['language'] = answer.lower()

  preferences['imdb_score'] = 10

  answer = input('Please write some keywords : ' )
  while not answer.replace(' ','').isalpha():
    answer = input('Please try again : ' )
  preferences['plot_keywords'] = answer.lower()

What is your prefered film genre? : thriller
What is your prefered content rating? : r
Do you prefer to watch Black and White films more than Color films? : no
What is your oprimal film duration in minutes? : 120
What is your prefered film language? : english
Please write some keywords : future


In [29]:
our_movie_dataset.loc[data_len] = preferences.values()

In [30]:
our_movie_dataset

,movie_title,genres,content_rating,color,duration,language,imdb_score,plot_keywords
0,Avatar,action adventure fantasy sci-fi,pg-13,color,178.0,english,7.9,avatar future marine native paraplegic
1,Pirates of the Caribbean: At World's End,action adventure fantasy,pg-13,color,169.0,english,7.1,goddess marriage ceremony marriage proposal pi...
2,Spectre,action adventure thriller,pg-13,color,148.0,english,6.8,bomb espionage sequel spy terrorist
3,The Dark Knight Rises,action thriller,pg-13,color,164.0,english,8.5,deception imprisonment lawlessness police offi...
4,John Carter,action adventure sci-fi,pg-13,color,132.0,english,6.6,alien american civil war male nipple mars prin...
...,...,...,...,...,...,...,...,...
4644,Circumstance,drama,r,color,107.0,persian,5.9,closeted lesbian iranian islamophobia propagan...
4645,Fruitvale Station,biography drama romance,r,color,85.0,english,7.5,22 year old b.a.r.t. bay area birthday police
4646,The Brass Teapot,comedy fantasy thriller,r,color,101.0,english,6.4,mini dress mini skirt money voyeur voyeurism
4647,Bambi,animation drama family,approved,color,70.0,english,7.4,animal bambi deer forest hunter


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

matrix = []

cv = CountVectorizer()

data_genge = np.array(cv.fit_transform(our_movie_dataset['genres']).todense())
data_color = np.array(cv.fit_transform(our_movie_dataset['color']).todense())
data_lang = np.array(cv.fit_transform(our_movie_dataset['language']).todense())
data_rating = np.array(cv.fit_transform(our_movie_dataset['content_rating']).todense())
data_keywords = np.array(cv.fit_transform(our_movie_dataset['plot_keywords']).todense())

In [32]:
our_movie_dataset.dtypes

movie_title        object
genres             object
content_rating     object
color              object
duration          float64
language           object
imdb_score        float64
plot_keywords      object
dtype: object

In [43]:
for i in range(data_len):
  our_movie_dataset.loc[i, 'distance'] = distance.cosine(data_genge[i], data_genge[data_len]) + distance.cosine(data_color[i], data_color[data_len]) \
  + 2*distance.cosine(data_lang[i], data_lang[data_len]) + distance.cosine(data_rating[i], data_rating[data_len]) + 3*distance.cosine(data_keywords[i], data_keywords[data_len]) \
  + abs(our_movie_dataset.loc[i, 'imdb_score'] - our_movie_dataset.loc[data_len, 'imdb_score']) \
  + abs(our_movie_dataset.loc[i, 'duration'] - our_movie_dataset.loc[data_len, 'duration'])

/usr/local/lib/python3.8/dist-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [44]:
print('Your preference')
print('Genge :', preferences['genres'])
print('Content rating :', preferences['content_rating'])
print('Color :', preferences['color'])
print('Duration :', preferences['duration'])
print('Language :', preferences['language'])
print('Plot keywords :', preferences['plot_keywords'])

Your preference
Genge : thriller
Content rating : r
Color : color
Duration : 120
Language : english
Plot keywords : future


**Films for you**

In [45]:
our_movie_dataset.sort_values(by=['distance'], ascending=True).head(15)

,movie_title,genres,content_rating,color,duration,language,imdb_score,plot_keywords,distance
1334,Serenity,action adventure sci-fi thriller,pg-13,color,119.0,english,8.0,alliance cannibalism future operative planet,5.211146
128,Mad Max: Fury Road,action adventure sci-fi thriller,r,color,120.0,english,8.1,australia desert escape on the run post apocal...,5.452786
1041,Chappie,action crime drama sci-fi thriller,r,color,120.0,english,6.9,artificial intelligence consciousness future m...,5.557858
2849,The Butterfly Effect,sci-fi thriller,r,color,120.0,english,7.7,blackout child pornography memory surprise end...,5.722650
4458,"Lock, Stock and Two Smoking Barrels",comedy crime,r,color,120.0,english,8.2,antique cockney accent hatchet money shotgun,5.800000
736,Collateral,crime drama thriller,r,color,120.0,english,7.6,city by night contract killer driving at night...,5.822650
3582,Platoon,drama war,r,color,120.0,english,8.1,1960s famous score hero vietnam vietnam war,5.900000
3004,Trade,crime drama thriller,r,color,120.0,english,7.5,friend girl mexico city new jersey sex,5.922650
2762,Slumdog Millionaire,drama romance,r,color,120.0,english,8.0,cheating falling from height murder non profes...,6.000000
3680,Do the Right Thing,drama,r,color,120.0,english,7.9,black militant frustration pizza pizza parlor ...,6.100000
